In [2]:
import os
import torch
import torch.nn as nn
import numpy as np

from thingsvision import get_extractor
from thingsvision.utils.storing import save_features
from thingsvision.utils.data import ImageDataset, DataLoader
from thingsvision.core.extraction import center_features

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from typing import Any, Dict, List

/home/boyanova/miniconda3/envs/feature_extract/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# path/to/images 
full_image_path = "/projects/archiv/DataStore_Boyanova/ExpAtt_EEG/Image_dataset/Images_plants" 
category = full_image_path.split("/")[-1].split("_")[1]

# path/to/output  
full_output_path =  f"/projects/archiv/DataStore_Boyanova/ExpAtt_EEG/Image_dataset/features_{category}"  
os.makedirs(full_output_path, exist_ok=True)

In [4]:
def extract_features(
                    extractor: Any,
                    module_name: str,
                    image_path: str,
                    out_path: str,
                    batch_size: int,
                    flatten_activations: bool,
                    apply_center_crop: bool,
                    class_names: List[str]=None,
                    file_names: List[str]=None,
) -> np.ndarray:
    """Extract features for a single layer."""                                    
    dataset = ImageDataset(
        root=image_path,
        out_path=out_path,
        backend=extractor.get_backend(),
        transforms=extractor.get_transformations(resize_dim=256, crop_dim=224),
        class_names=class_names,
        file_names=file_names,
    )
    batches = DataLoader(dataset=dataset, batch_size=batch_size, backend=extractor.get_backend())
    features = extractor.extract_features(
                    batches=batches,
                    module_name=module_name,
                    flatten_acts=flatten_activations,
    )
    
    
    # save features to disk
    save_features(features, out_path=f'{out_path}/raw_{model_name}_{module_name}', file_format='npy')
    return features

In [25]:
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 32 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
class_names = None  # optional list of class names for class dataset
file_names = None # optional list of file names according to which features should be sorted
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# alexnet, vgg16, resnet50 source = 'torchvision
# cornet-s and cornet-rt source = 'custom'  
model_name = 'cornet-s'
source = 'custom'
extractor = get_extractor( 
            model_name=model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            source=source)

# custom extraction // 
# after every layer extraction + pca result is saved in a folder in the output_dir
module_names = []
for name, layer in extractor.model.named_modules(): 
    if isinstance(layer, torch.nn.Conv2d):
        module_names.append(name)
        
print(module_names)
if model_name == "cornet-s":
    module_names = ['V1.conv1',  'IT.conv3']
        



Using device: cuda

['V1.conv1', 'V1.conv2', 'V2.conv_input', 'V2.skip', 'V2.conv1', 'V2.conv2', 'V2.conv3', 'V4.conv_input', 'V4.skip', 'V4.conv1', 'V4.conv2', 'V4.conv3', 'IT.conv_input', 'IT.skip', 'IT.conv1', 'IT.conv2', 'IT.conv3']


In [26]:
module_names

'IT.conv3'

In [12]:
module_names

['classifier.1', 'classifier.4', 'classifier.6']

In [ ]:
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 32 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
class_names = None  # optional list of class names for class dataset
file_names = None # optional list of file names according to which features should be sorted
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# alexnet, vgg16, resnet50 source = 'torchvision
# cornet-s and cornet-rt source = 'custom'  
model_name = 'cornet-s'
source = 'custom'
extractor = get_extractor( 
            model_name=model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            source=source)

# custom extraction // 
# after every layer extraction + pca result is saved in a folder in the output_dir
module_names = []
for name, layer in extractor.model.named_modules():
    if isinstance(layer, torch.nn.Conv2d):
            module_names.append(name)
 
# in case your code breaks for some reason 
# when you restart you can adjust from which layer 
# by picking the elements from the list module_names  
for idx, l in enumerate(module_names):
  module_name = l
  print(module_name)
  print('Layer ', idx, 'out of', len(module_names))
  print("-" * 20) 
  
  extract_features(
                    extractor = extractor,
                    module_name =  module_name,
                    image_path=full_image_path,
                    out_path=full_output_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    class_names=class_names,
                    file_names=file_names,
)